In [1]:
## IMPORT ALL LIBRAIES...

import os
import copy
import pandas as pd 
import numpy as np 
import torch
import albumentations as A
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import torch.nn as nn 
from albumentations.pytorch.transforms import ToTensorV2
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision.transforms import Compose
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader 
import torchvision.models as models
from ImageProcess import SplitImages, checkingsplitsame
from ImageClfModeling import set_parameter_requires_grad, train_model

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# test image path
test_imagepath = "TestImage/test/"

In [4]:
# global transformation into tensor form (test data)
global_transform = Compose([transforms.ToTensor()])
database_test = ImageFolder(root=test_imagepath, transform=global_transform)

In [5]:
#--------------------------------------------CLASS LABELS TO INDEX---------------------------------------------

# class names from index to class
class_names = [x for x in database_test.classes]
class_name_dict = {}
for ix, val in enumerate(class_names):
    class_name_dict[ix] = val

In [6]:
test_sz = len(database_test)
# test
test_ds = random_split(database_test, [test_sz])[0]

In [7]:
# test data transformation 
print()
print("AUGMENTING TEST IMAGES")
print()
test_transform = A.Compose([A.Resize(224, 244),
                            ToTensorV2()])

testdata_trans = []

for img, lb in tqdm(test_ds):
    im = test_transform(image = img.permute(1,2,0).numpy())
    testdata_trans.append((im["image"], lb))


AUGMENTING TEST IMAGES



  0%|          | 0/3068 [00:00<?, ?it/s]

In [8]:
#--------------------------------------------DATA LOADERS---------------------------------------------  


# batch size for test data (we want to test all the data at one go)
batch_test = len(test_ds)

# Iterative Data loader
test_dl = DataLoader(testdata_trans, batch_size=batch_test, shuffle=False, num_workers=2)

In [9]:
def initialize_model(num_classes, feature_extract, use_pretrained=True):
    model_ft = models.vgg16(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] =  nn.Linear(num_ftrs,num_classes)
    return model_ft

In [10]:
#--------------------------------------INITIALIZING PRE TRAINED MODEL-----------------------------------


num_classes = len(class_name_dict)
feature_extract = False

# pre trained model
model_ft = initialize_model(num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

/usr/lib/python3/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/lib/python3/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/ubuntu/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
#--------------------------------SENDING MODEL TO GPU-------------------------------------


# Send the model to GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.benchmark = True
print(f'using {device} device')

model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")

for name,param in model_ft.named_parameters():
    if param.requires_grad == True:
        print("\t",name)

using cuda device
Params to learn:
	 features.0.weight
	 features.0.bias
	 features.2.weight
	 features.2.bias
	 features.5.weight
	 features.5.bias
	 features.7.weight
	 features.7.bias
	 features.10.weight
	 features.10.bias
	 features.12.weight
	 features.12.bias
	 features.14.weight
	 features.14.bias
	 features.17.weight
	 features.17.bias
	 features.19.weight
	 features.19.bias
	 features.21.weight
	 features.21.bias
	 features.24.weight
	 features.24.bias
	 features.26.weight
	 features.26.bias
	 features.28.weight
	 features.28.bias
	 classifier.0.weight
	 classifier.0.bias
	 classifier.3.weight
	 classifier.3.bias
	 classifier.6.weight
	 classifier.6.bias


In [12]:
#--------------------------------MODEL PARAMETERS---------------------------------------------


# Epochs
epochs_ = 12

# learning rate
lr= 1e-3

# momentum
moment = 0.9

# Optimizer
optimizer_ft = torch.optim.SGD(params_to_update, lr=lr, momentum=moment)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

In [13]:
# Load model

model_ft.load_state_dict(torch.load("VggNet_FER_weight.pt"))

<All keys matched successfully>

In [14]:
def topk_analysis(outputs, k, allbreeds):
    outputs = outputs.to("cpu")
    predval, predix = torch.sort(outputs, descending=True)
    return [allbreeds.loc[x.item()]["Label_name"] for x in predix[0][:k]]

In [21]:
cls_name = {0:"Surprise", 1:"Fear", 2:"Disgust", 3:"Happiness",
            4:"Sadness", 5:"Anger", 6:"Neutral"}

In [16]:
scene_name = pd.read_csv("fer_names.csv")

In [28]:
#-----------------------------TEST IMAGES-----------------------

# input data 
actual_label = []
pred_label = []
for inputs, lbs in test_dl:
    for i in tqdm(range(3068)):
        input_dl = DataLoader([inputs[i]], batch_size=1, shuffle=False, num_workers=2)
        lbs_x = cls_name[int(lbs[i])]
        for inputs_x in input_dl:
            inputs_x = inputs_x.to(device) 
            model_ft.eval()
            with torch.no_grad():
                outputs_ = model_ft(inputs_x)
                actual_label.append(lbs_x)
                pred_label.append(topk_analysis(outputs_,3,scene_name))

  0%|          | 0/3068 [00:00<?, ?it/s]

In [29]:
compare_df = pd.DataFrame(data = pred_label, columns=["Top 1", "Top 2", "Top 3"])
compare_df["Actual Label"] = actual_label

In [50]:
# top 3 accuracy and classificaion accuracy by class

# classification by class accuracy
cls_acc = {"Surprise":0, "Fear":0, "Disgust":0, "Happiness":0,
            "Sadness" :0,"Anger":0, "Neutral":0}
counter = 0
for i, j in zip(actual_label, pred_label):
    if i in j:
        counter+=1
        cls_acc[i]+=1

for k in cls_acc.keys():
    cls_acc[k] = (cls_acc[k]/(len(compare_df[compare_df["Actual Label"]==k])))*100
    
top3_acc = round((counter/len(database_test))*100, 4)
top3_acc

92.8618

In [58]:
cls_by_acc = pd.DataFrame(index = list(cls_acc.keys()), data=list(cls_acc.values()), columns=["Accuracy %"])
cls_by_acc

,Accuracy %
Surprise,88.449848
Fear,71.621622
Disgust,78.750000
Happiness,95.611814
Sadness,97.698745
Anger,75.308642
Neutral,96.617647
